<center>
</center> 

# Введение в программирование на Python 
### Занятие 5. Потоки ввода-вывода, неймспейсы и декораторы

<br />
<br />
Александр Авдюшенко <br />
1 октября 2020

<center><img src="./images/input_perf.jpg" width=530 style="display: inline-block;">&nbsp;<img src="./images/input_perfolenta.png" width=500 style="display: inline-block;"></center> 


# Потоки ввода-вывода

## Откуда берутся потоки

In [45]:
print('Hello!')  # How does it work?

Hello!


<div align="center">
<img alt="streams" src="http://maxvoloshin.com/images/shell-streams.png" width="600px" />
</div>

Подробности на [xgu.ru](http://xgu.ru/wiki/Стандартные_потоки_ввода/вывода)

In [54]:
import sys

sys.stdout.write('Hello again!\n')

Hello again!


In [56]:
sys.stderr.write('Danger!\n')

Danger!

### Каждый байт сообщения печатается сразу?

```python
import sys
from time import sleep


print('test', end='')
print('err', file=sys.stderr)

sleep(1)
```

Нет, потому что вывод буферизирован, и обычно ожидает конец строки `\n`, либо явной команды на опустошение буфера.

```python
import sys
from time import sleep

print('test', end='')
sys.stdout.flush()  # или просто print('test', end='', flush=True)
print('err', file=sys.stderr)
sleep(1)
```

### Откуда ещё берутся потоки?

#### Файлы

In [2]:
f = open('../../README.md', encoding='utf-8')
f

<_io.TextIOWrapper name='../../README.md' mode='r' encoding='utf-8'>

In [3]:
content = f.read()

In [7]:
type(content)

str

In [24]:
len(content)

2340

In [25]:
f.close()  # implicit f.flush()

Читать весь файл в память может быть очень дорого

In [8]:
f = open('../../README.md', encoding='utf-8')

In [9]:
f.readline()

'# Введение в программирование на Python — 2020\n'

А если там не строки? Или нужно прочитать сразу пачку строк?

In [11]:
chunk_size = 512
f.read(chunk_size)

'* Пример подготовки даных и обучения градиентного бустинга на Python.  \n\n## Установка интерпретатора и среды разработки\n\n### Как открыть на ноутбуке лекции\n[поставить Jupyter Notebook](https://jupyter.readthedocs.io/en/latest/install.html)\n\n### Как удобно писать код\n* [Среда разработки PyCharm](https://www.jetbrains.com/pycharm/)\n* [Среда разработки VSCode](https://code.visualstudio.com/)\n* [Текстовый редактор Atom](https://atom.io/)\n\n## Полезные ссылки\n* [Хороший учебник «А Byte of Python» (русский перевод'

In [12]:
help(f.read)

Help on built-in function read:

read(size=-1, /) method of _io.TextIOWrapper instance
    Read at most n characters from stream.
    
    Read from underlying buffer until we have n characters or we hit EOF.
    If n is negative or omitted, read until EOF.



In [13]:
f.close()

Окей, хотим прочитать чанками до конца...

In [30]:
f = open('../../README.md', encoding='utf-8')

In [31]:
lines = 0
while True:
    chunk = f.read(chunk_size)
    if not chunk:
        break
    lines += chunk.count('\n')
lines

50

In [32]:
f.read()

''

In [33]:
f.close()

А если нужно вернуться в начало прочитанного файла? Обязательно открывать заново?

In [25]:
f = open('../../README.md', encoding='utf-8')
content = f.read()

In [26]:
f.read()

''

In [27]:
import io
f.seek(0, io.SEEK_SET)

0

In [28]:
f.readline()

'# Введение в программирование на Python — 2020\n'

In [29]:
help(f.tell)

Help on built-in function tell:

tell() method of _io.TextIOWrapper instance
    Return current stream position.



In [30]:
f.tell()

77

Хочу читать с конца файла!

In [43]:
f.seek(-100, io.SEEK_END)  # упс...

UnsupportedOperation: can't do nonzero end-relative seeks

In [44]:
f.close()

### Modes

In [46]:
f = open('../../README.md', 'r', encoding='utf-8')

f.readline()  # в текстовом режиме читаются строки (strings)

'# Введение в программирование на Python — 2020\n'

In [48]:
f = open('../../README.md', 'rb')
f.readline()  # в байтовом (бинарном) режиме читаются байты

b'# \xd0\x92\xd0\xb2\xd0\xb5\xd0\xb4\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xb2 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb3\xd1\x80\xd0\xb0\xd0\xbc\xd0\xbc\xd0\xb8\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xbd\xd0\xb0 Python \xe2\x80\x94 2020\r\n'

<div align="center">
<img alt="random-access" src="https://im0-tub-ru.yandex.net/i?id=81194f1ca168f04af2ce3a77b0f131e3&n=13" width="600px" />
</div>

In [49]:
f.seek(-99, io.SEEK_END)  # только для байтовых строк

3571

In [50]:
f.read()

b'\xd1\x81\xd0\xba\xd0\xbe\xd0\xb2](https://classroom.github.com/a/TbqBYgcY), \xd0\xb4\xd0\xb5\xd0\xb4\xd0\xbb\xd0\xb0\xd0\xb9\xd0\xbd 4 \xd0\xbe\xd0\xba\xd1\x82\xd1\x8f\xd0\xb1\xd1\x80\xd1\x8f, 20:00 \xd0\x9c\xd0\xa1\xd0\x9a\r\n'

### Write

In [11]:
f = open('tmp.txt', 'w')

In [12]:
f.write('test\n')

5

In [13]:
f.close()

In [14]:
f = open('tmp.txt', 'a')
f.write('another one\n')
f.close()

In [15]:
f = open('tmp.txt')
f.read()

'test\nanother one\n'

In [16]:
f.close()

In [17]:
f = open('tmp.bin', 'wb')

In [18]:
f.write(b'\x20\x12')

2

In [19]:
f.close()

### Read + write

In [24]:
import io
f = open('tmp.txt', 'r+')

In [25]:
f.write('diff\n')

5

In [26]:
f.seek(0, io.SEEK_SET)
f.read()

'diff\nanother one\n'

In [27]:
f.close()

### Context manager

In [28]:
with open('tmp.txt') as f:
    print(f.read())

diff
another one



## In-memory streams

### StringIO

In [66]:
import io

In [67]:
output = io.StringIO()
output.write('This goes into the stream. ')
print('And so does this.', file=output)

In [68]:
print(output.getvalue())

This goes into the stream. And so does this.



In [69]:
output.close()  # отбрасываем содержимое стрима

In [70]:
input_ = io.StringIO('Inital value for read stream')

In [71]:
print(input_.read())

Inital value for read stream


### BytesIO

In [72]:
output = io.BytesIO()
output.write('This goes into the stream. '.encode('utf-8'))
output.write('ÁÇÊ'.encode('utf-8'))

6

In [73]:
print(output.getvalue())

b'This goes into the stream. \xc3\x81\xc3\x87\xc3\x8a'


In [46]:
input_ = io.BytesIO(b'Inital value for read stream')

In [47]:
print(input_.read())

b'Inital value for read stream'


# Пространства имен [Namespaces]

### Локальный namespace

In [1]:
def show_scope(x):
    y = " World!"
    print(locals())

show_scope("Hello")

{'x': 'Hello', 'y': ' World!'}


### Глобальный namespace

In [2]:
def show_scope(x):
    y = " World!"
    print(globals())

show_scope("Hello")

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'def show_scope(x):\n    y = " World!"\n    print(locals())\n\nshow_scope("Hello")', 'def show_scope(x):\n    y = " World!"\n    print(globals())\n\nshow_scope("Hello")'], '_oh': {}, '_dh': ['C:\\Users\\avalur\\avalur\\ismc\\python-2020\\lessons\\05'], 'In': ['', 'def show_scope(x):\n    y = " World!"\n    print(locals())\n\nshow_scope("Hello")', 'def show_scope(x):\n    y = " World!"\n    print(globals())\n\nshow_scope("Hello")'], 'Out': {}, 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x000002737EC78B88>>, 'exit': <IPython.core.autocall.ZMQExitAutocall object at 0x000002737ED37B48>, 'quit': <IPython.core.autocall.ZMQExitAutocall object at 0x000002737ED37

In [3]:
locals() == globals()

True

### Встроенный namespace (\_\_builtins\_\_)

In [4]:
'range' in globals()

False

In [5]:
'range' in dir(__builtins__)

True

### Функции создают свой namespace

In [6]:
def function():
    inner_variable = 42

function()
inner_variable

NameError: name 'inner_variable' is not defined

In [7]:
def function(arg):
    print(locals())
    print(locals() == globals())

function(1)

{'arg': 1}
False


### Циклы и условия не создают свой namespace

In [8]:
for i in range(3):
    in_for = i

print(in_for)
print(i)

2
2


In [9]:
if True:
    in_if = 2
    
print(in_if)

2


### Генераторы создают namespace

In [10]:
i = 'Hello'
[i for i in range(10)]
print(i)

Hello


### Правило LEGB

<center>
    <img src="./images/LEGB_Python.png" width=600 style="display: inline-block;">
</center> 


In [11]:
global_var = 'global_var'

def func(): 
    local_var = 'local_var'
    # global_var is in global namespace
    print('func:', global_var)
    print('func:', local_var)

func()
print(global_var)
print(local_var)

func: global_var
func: local_var
global_var


NameError: name 'local_var' is not defined

In [12]:
global_var = 'global_var'

def func():
    global_var = 'global_var_modified'
    # global_var shadows another variable with same name
    print('func  :', global_var)

func()
print('global:', global_var)

func  : global_var_modified
global: global_var


### global 

In [13]:
global_var = 'global_var'

def func():
    global global_var
    global_var = 'global_var_modified'
    print(global_var)

func()
print(global_var)

global_var_modified
global_var_modified


<div class="alert alert-danger">
<b>Антипаттерн: </b> использование global
</div>

In [14]:
import dis
global_var = 'global_var'

def func():
    global global_var
    global_var = 'global_var_modified'
    print(global_var)

func()
print(global_var)
dis.dis(func)

global_var_modified
global_var_modified
  6           0 LOAD_CONST               1 ('global_var_modified')
              2 STORE_GLOBAL             0 (global_var)

  7           4 LOAD_GLOBAL              1 (print)
              6 LOAD_GLOBAL              0 (global_var)
              8 CALL_FUNCTION            1
             10 POP_TOP
             12 LOAD_CONST               0 (None)
             14 RETURN_VALUE


In [15]:
def func():
    global new_global_var
    new_global_var = 'new_global_var'
    print(new_global_var)

func()
print(new_global_var)

new_global_var
new_global_var


### Вложенные функции

In [29]:
def outer():
    outer_var = 'foo'

    def inner():
        inner_var = 'bar'
        print('inner:', outer_var)
        print('inner:', inner_var)

    inner()

    print('outer:', outer_var)
    print('outer:', inner_var)


In [30]:
outer()

inner: foo
inner: bar
outer: foo


NameError: name 'inner_var' is not defined

### Замечание
Функции имеют доступ к внешним пространствам имён относительно того места где они были **определены**, а не **вызваны**

In [3]:
def print_iter():
    print(iter_)

def test(func):
    for iter_ in range(3):
        func()

test(print_iter)

NameError: name 'iter_' is not defined

In [5]:
def print_iter():
    print(iter_)

def test(func):
    for iter_ in range(3):
        func()

test(print_iter)
iter_ = 'global_iter'


global_iter
global_iter
global_iter


### Проблема

In [18]:
def outer():
    var = 'outer'

    def inner():
        global var
        var = 'inner'
        print('inner :', var)

    inner()    
    print('outer :', var)

outer()
print('global:', var)

inner : inner
outer : outer
global: inner


In [19]:
# cleanup
del var

### Решение : nonlocal

In [20]:
def outer():
    var = 'outer'

    def inner():
        nonlocal var
        var = 'inner'
        print('inner :', var)

    inner()    
    print('outer :', var)

outer()
print('global:', var)

inner : inner
outer : inner


NameError: name 'var' is not defined

<div class="alert alert-danger">
<b>Антипаттерн: </b> использование nonlocal
</div>

### Другое странное решение

In [21]:
def outer(): 
    outer.var = 'v1'

    def inner():
        outer.var = 'v2'
        print('inner:', outer.var)

    inner()
    print('outer:', outer.var)


outer()
print(outer.var)

inner: v2
outer: v2
v2


### Где теперь лежит переменная?

In [22]:
def outer():
    nonlocal_var = 'v1'

    def inner():
        print('inner:', nonlocal_var)

    return inner


f = outer()
f()

inner: v1


## Замыкания [Closures]
_In computer programming languages, a closure is a function together with a referencing environment of that function. A closure function is any function that uses a variable that is defined in an environment (or scope) that is external to that function, and is accessible within the function when invoked from a scope in which that free variable is not defined._

Существования замыканий следует из правила LEGB и возможностью оперировать с функциями как обьектами.

In [23]:
def make_adder(x):
    def adder(y):
        return x + y
    return adder

In [24]:
add_two  = make_adder(2)
add_five = make_adder(5)

add_two(7) + add_five(10)

24

In [25]:
make_adder(2)(7)

9

<div class="alert alert-danger">
<b>Антипаттерн: </b> использование сложных замыканий
</div>

In [26]:
def value_factory(value=10):
    def get_value():
        return value

    def set_value(new_value):
        nonlocal value
        value = new_value
        return value
    return get_value, set_value


get_value, set_value = value_factory()
print(get_value())

set_value(10**6)

print(get_value())

10
1000000


In [27]:
def make_adder(x):

    def add(y):
        return x + y

    def update(new_x):
        nonlocal x
        x = new_x

    add.update = update
    return add

adder = make_adder(10)
print(adder(10))

adder.update(100)

print(adder(10))

20
110


## Декораторы

In [29]:
import sys

def deprecate(func):
    def inner(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return inner

pprint = deprecate(print)

pprint([1, 2, 3])

[1, 2, 3]


print is deprecated


### Синтаксис декораторов

In [30]:
import sys

def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def add(x, y):
    return x + y

# add = deprecated(add)

add(1, 2)

add is deprecated


3

### Pusheenize

In [31]:
from IPython import display

def pusheenize(func):
    return display.HTML('<img src="https://media1.tenor.com/images/4a950a1e221d93e654047ecee711af5a/tenor.gif">')

@pusheenize
def dummy_function(arg):
    print(arg)

dummy_function

### Проблема

In [32]:
@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

wrapper
None


### Решение 1

In [33]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


### Решение 2

In [34]:
import functools

def deprecated(func):
    @functools.wraps(func) 
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


![dicaprio](https://camo.githubusercontent.com/68cfb12d9c144fec645a61fa64151b35efc70f68/687474703a2f2f69302e6b796d2d63646e2e636f6d2f70686f746f732f696d616765732f6f726967696e616c2f3030302f3338342f3137362f6432662e6a7067)

In [35]:
import functools
import sys

def trace(dest=sys.stderr):
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(
                '{} called with args {}, kwargs {}!'
                .format(func.__name__, args, kwargs),
                file=dest
            )
            return func(*args, **kwargs)
        return wrapper
    return wraps


@trace(dest=sys.stderr)
def f(x, test):
    if test > 1:
        return f(x, test / 2)


f('Hi!', test=42)

f called with args ('Hi!',), kwargs {'test': 42}!
f called with args ('Hi!', 21.0), kwargs {}!
f called with args ('Hi!', 10.5), kwargs {}!
f called with args ('Hi!', 5.25), kwargs {}!
f called with args ('Hi!', 2.625), kwargs {}!
f called with args ('Hi!', 1.3125), kwargs {}!
f called with args ('Hi!', 0.65625), kwargs {}!


### Декоратор Once

In [2]:
def once(func):
    called = False

    def wrapper(*args, **kwargs):
        nonlocal called
        if not called:
            called = True
            return func(*args, **kwargs)

    return wrapper


@once
def f():
    print('Hi!')

f()
f()

Hi!


### Цепочки декораторов

In [37]:
@deprecated
@trace()
def f(x):
    return x

f(1)

f is deprecated!
f called with args (1,), kwargs {}!


1

# Всем большое спасибо и успехов в преподавании Питона!